In [ ]:
import sys

print(sys.version)
%load_ext autoreload
%autoreload 2

In [ ]:
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

saes, sparsities = get_gpt2_res_jb_saes(device='mps')

In [ ]:
from sae_lens.training.evals import run_evals
from sae_lens.training.activations_store import ActivationsStore
import transformer_lens
from tests.unit.helpers import build_sae_cfg
from datasets import Dataset
import rich

model = transformer_lens.HookedTransformer.from_pretrained("gpt2-small", device="mps")
cfg = build_sae_cfg(
    checkpoint_path="./checkpoints",
    train_batch_size=32,
    total_training_tokens=100,
    context_size=8,
    device="mps"
)
dataset = Dataset.from_list([{"text": "hello world"}] * 1000)
activation_store = ActivationsStore.from_config(model, cfg, dataset=dataset)
for name, sae in saes.items():
    res = run_evals(sae, activation_store, model, 1, f"_{name}")
    rich.print(res)
    break